In [1]:
!nvidia-smi

Wed Feb  1 18:50:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   39C    P0    52W / 300W |      0MiB / 32508MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-02-01 18:50:54.697127: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 18:50:54.860759: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14199316374074684166
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 32476168192
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7807855643240213277
physical_device_desc: "device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0"
xla_global_id: 416903419
]


2023-02-01 18:50:57.964178: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 18:51:02.086765: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0


In [4]:
import gc
import math
import gdown
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import tqdm.notebook as tqdm

In [5]:
aspect = ['molecular','function']
aspect_abbr = 'mf'

In [6]:
file = 'golabeler_train/mf_df.csv'
# if not os.path.exists(file):
#     url = "https://drive.google.com/file/d/1RyeLQPFTMWAIr-OzELTWIx60ln-mZ7g_/view?usp=sharing"
#     output = file
#     gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [7]:
df = pd.read_csv(file)
df

,index,Entry,accessions,Sequence,Gene Ontology (molecular function),interpros,orgs
0,240,128UP_DROME,P32234; Q9V648;,MSTILEKISAIESEMARTQKNKATSAHLGLLKAKLAKLRRELISPK...,GO:0036094;GO:0043167;GO:0032553;GO:0017076;GO...,"['IPR031167', 'IPR031662', 'IPR006074', 'IPR00...",7227
1,261,14310_ARATH,P48347; Q9LME5;,MENEREKQVYLAKLSEQTERYDEMVEAMKKVAQLDVELTVEERNLV...,GO:0036094;GO:0043167;GO:0032553;GO:0017076;GO...,"['IPR000308', 'IPR023409', 'IPR023410']",3702
2,263,14311_ARATH,Q9S9Z8; A0JQ87; F4HWN0; Q0WL19;,MENERAKQVYLAKLNEQAERYDEMVEAMKKVAALDVELTIEERNLL...,GO:0051117;GO:0005515;GO:0005488;GO:0019899;GO...,"['IPR000308', 'IPR023409', 'IPR023410']",3702
3,287,14333_ARATH,P42644; F4KBI7; Q945L2;,MSTREENVYMAKLAEQAERYEEMVEFMEKVAKTVDVEELSVEERNL...,GO:0036094;GO:0043167;GO:0032553;GO:0017076;GO...,"['IPR000308', 'IPR023409', 'IPR023410']",3702
4,296,14335_ARATH,P42645;,MSSDSSREENVYLAKLAEQAERYEEMVEFMEKVAKTVETEELTVEE...,GO:0036094;GO:0043167;GO:0032553;GO:0017076;GO...,"['IPR000308', 'IPR023409', 'IPR023410']",3702
...,...,...,...,...,...,...,...
34483,550264,ZYM1_SCHPO,Q9UTC0;,MEHTTQCKSKQGKPCDCQSKCGCQDCKESCGCKSSAVDNCKCSSCK...,GO:0043167;GO:0046914;GO:0043169;GO:0046872;GO...,['IPR001008'],284812
34484,550266,ZYX_CAEEL,Q9U3F4; H2L2F5; H2L2F6; H2L2F7; Q9U3F5;,MGPPPPPPPPPLLPSGEILPSRKWKTEDAPRRNNHPAPAPPKPSRP...,GO:0003674;GO:0005515;GO:0005488;GO:0019899,['IPR001781'],6239
34485,550267,ZYX_CHICK,Q04584;,MASPGTPGTRMTTTVSINISTPSFYNPQKKFAPVVAPKPKVNPFKT...,GO:0003674;GO:0008092;GO:0005515;GO:0005488,['IPR001781'],9031
34486,550268,ZYX_HUMAN,Q15942; A4D2G6; B4DQX7; Q6I9S4;,MAAPRPSPAISVSVSAPAFYAPQKKFGPVVAPKPKVNPFRPGDSEP...,GO:0003676;GO:0005488;GO:0003723;GO:0003674;GO...,['IPR001781'],9606


In [8]:
go_terms_bp = set()
for idx, row in df.iterrows():
    for term in row['Gene Ontology ('+' '.join(aspect)+')'].split(';'):
        go_terms_bp.add(term)
go_terms_bp = list(go_terms_bp)
go_terms_bp.sort()
print(len(go_terms_bp))
# print(go_terms_bp)

652


In [10]:
def get_segments(sequence,segment_size=100,gap=30):
    segments = []
    start = 0
    end = segment_size
    while end <= len(sequence):
        segments.append(sequence[start:end])
        start += gap
        end += gap
    last_segment = sequence[start:]
    segments.append(last_segment)
    
    return segments

def get_training_data(df,segment_size=100,gap=30):
    training_data = list()
    for idx,row in tqdm.tqdm(df.iterrows()):
        labels = [0] * len(go_terms_bp)
        for term in row['Gene Ontology ('+' '.join(aspect)+')'].split(';'):
            labels[go_terms_bp.index(term)] = 1
        segments = get_segments(row['Sequence'],segment_size,gap)
        for segment in segments:
            training_data.append([row['Entry'],segment,labels])
    return training_data

In [11]:
training_data = get_training_data(df,gap=30)
print(len(training_data))

0it [00:00, ?it/s]

574962


In [12]:
def get_ngrams(segment,n=3):
    ngrams = []
    for i in range(len(segment)-n+1):
        ngrams.append(segment[i:i+n])
    return ngrams

In [13]:
# Generate training data of ngrams
# if os.path.exists('bp/training_data_4grams.npy'):
#     print('Loading saved ngrams...')
#     training_data_ngrams = np.load('bp/training_data_4grams.npy',allow_pickle=True)
# else:
print('Preparing from scratch...')
training_data_ngrams = []

for i in tqdm.tqdm(range(len(training_data))):
    training_data_ngrams.append([training_data[i][0],get_ngrams(training_data[i][1],n=4),training_data[i][2]])
        
#     np.save('bp/training_data_4grams.npy',training_data_ngrams)
    
print(len(training_data_ngrams))

Preparing from scratch...


  0%|          | 0/574962 [00:00<?, ?it/s]

574962


In [14]:
def get_skip_grams(segment,skip=1,n=3):
    skip_grams = []
    window_size = skip + n
    for i in range(len(segment)-window_size+1):
        window = segment[i:i+window_size]
        indices = list(range(window_size))
        indices.pop(0)
        for idx in indices[::-1]:
            temp = ''
            for j in range(window_size):
                if j!=idx:
                    temp+=window[j]
            skip_grams.append(temp)

    return skip_grams

In [15]:
# if os.path.exists('bp/training_data_skip1_4grams.npy'):
#     print('Loading saved skip grams...')
#     training_data_skip_grams = np.load('bp/training_data_skip1_4grams.npy',allow_pickle=True)
# else:
print('Preparing from scratch...')
training_data_skip_grams = []
for i in tqdm.tqdm(range(len(training_data))):
    training_data_skip_grams.append([training_data[i][0],get_skip_grams(training_data[i][1],n=4),training_data[i][2]])
#     np.save('bp/training_data_skip1_4grams.npy',training_data_skip_grams)
print(len(training_data_skip_grams))

Preparing from scratch...


  0%|          | 0/574962 [00:00<?, ?it/s]

574962


In [16]:
import tensorflow as tf
import tensorflow_addons as tfa

In [17]:
#Assuming training_data as global variable

def train_test_split(X,y,fold_no,prev_index,Kfolds=5):
    test_split = 1/Kfolds
    
    start_index = prev_index
    end_index = (fold_no + 1) * (test_split) * len(X)
    end_index = round(end_index)
    
    if end_index==len(X):
        end_index -= 1
    
    entry = training_data[end_index][0]
    entries = [sample[0] for sample in training_data]
    
    first_occurence = entries.index(entry)
    entries.reverse()
    
    last_occurence = entries.index(entry)
    last_occurence = len(entries) - last_occurence - 1
    
    del entries
    gc.collect()
    
    end_index = first_occurence if (abs(end_index-first_occurence) < abs(end_index-last_occurence)) else last_occurence
    
    X_test = X[start_index:end_index+1]
    y_test = y[start_index:end_index+1]
    X_train = X[:start_index]
    X_train.extend(X[end_index+1:])
    y_train = y[:start_index]
    y_train.extend(y[end_index+1:])
    
    return X_train, y_train, X_test, y_test, start_index, end_index + 1

In [18]:
# MAX_WORDS = 13824
# MAX_LEN_NG = 98 #100
# MAX_LEN_SG = 291 #300
MAX_WORDS = 331776
MAX_LEN_NG = 97 #100
MAX_LEN_SG = 384 #300
# MAX_WORDS = 7962624
# MAX_LEN_NG = 96 #100
# MAX_LEN_SG = 475 #300

def tokenization(X_train,X_test,maxlen):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS)
    tokenizer.fit_on_texts(X_train)

    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

    vocab_size = len(tokenizer.word_index) + 1

    X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=maxlen)
    
    return X_train, X_test, vocab_size, tokenizer

In [19]:
from tensorflow.keras import backend as K

false_negative_penalty = 6
false_positive_penalty = 1

def custom_loss(y_true, y_logit):

    loss = float(0)
    y_true = tf.cast(y_true, tf.float32)
    y_logit = tf.cast(y_logit, tf.float32)
    
    first_term = false_negative_penalty * float(y_true) * - K.log(y_logit + K.epsilon())
    second_term = false_positive_penalty * (1 - float(y_true)) * - K.log(1 - y_logit + K.epsilon())
    
    loss = K.mean(first_term+second_term)

    return loss

def precision(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=1)
    precision = true_positives / (predicted_positives + K.epsilon())
    
    return K.mean(precision)


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=1)
    recall = true_positives / (possible_positives + K.epsilon())
    return K.mean(recall)

def f1_score(y_true, y_pred):
    rec = recall(y_true,y_pred)
    prec = precision(y_true,y_pred)
    f1 = 2*prec*rec/(prec+rec)
    return f1

In [20]:
class attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences=True,**kwargs):
        self.return_sequences = return_sequences
        super(attention,self).__init__()

    def get_config(self):
      config = super().get_config().copy()
      config.update({
          'return_sequences': self.return_sequences 
      })
      return config
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [21]:
NUM_CLASSES = 652 #For bp (Change according to aspects)

def get_model_ng_sg(vocab_size_ng, vocab_size_sg):
    #Input layers

    input_ngrams = tf.keras.layers.Input(shape=(MAX_LEN_NG,)) 
    input_skip_grams = tf.keras.layers.Input(shape=(MAX_LEN_SG,)) 

    #embeddings
    embedding_layer_ngrams = tf.keras.layers.Embedding(vocab_size_ng, 32)(input_ngrams)
    embedding_layer_skip_grams = tf.keras.layers.Embedding(vocab_size_sg, 32)(input_skip_grams)

    #BI-LSTMs for each of the inputs
    sequence_output_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(embedding_layer_ngrams)
    attention_output_1 = attention(return_sequences=False)(sequence_output_1)
    dropout_1 = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_1)

    sequence_output_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(embedding_layer_skip_grams)
    attention_output_2 = attention(return_sequences=False)(sequence_output_2)
    dropout_2 = tf.keras.layers.Dropout(0.3)(attention_output_2)
    dense_layer_2 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_2)

    max_layer = tf.keras.layers.Maximum()([dense_layer_1,dense_layer_2])

    model = tf.keras.models.Model(
        inputs=[
            input_ngrams,
            input_skip_grams
        ], 
        outputs=max_layer)
    
    return model

### Model using skip grams and ngrams

In [22]:
#Considering ngrams and skip grams
X_train_ng = [' '.join(sample[1]) for sample in training_data_ngrams]
X_train_sg = [' '.join(sample[1]) for sample in training_data_skip_grams]
y_train = [sample[2] for sample in training_data]

del training_data_ngrams
del training_data_skip_grams

In [23]:
print(len(X_train_ng),len(X_train_sg))

574962 574962


In [24]:
test_df = pd.read_csv('golabeler_test/mf_df.csv')
test_df

,level_0,index,Entry,accessions,Sequence,Gene Ontology (molecular function),interpros,orgs
0,181,1028,3BHS7_MOUSE,Q9EQC1; A2RTR5;,MADSAQVPTLVYLVTGGCGFLGEHIVRMLLEREPRLRELRVFDLHL...,GO:0016491;GO:0003824;GO:0003674;GO:0016616;GO...,"['IPR002225', 'IPR016040']",10090
1,183,1043,3BP1_MOUSE,P55194; E9QMQ2; Q99KK8;,MAESFKELDPDSSMGKALEMTCAIQNQLARILAEFEMTLERDVLQP...,GO:0017124;GO:0005515;GO:0005488;GO:0003674;GO...,"['IPR027267', 'IPR004148', 'IPR008936', 'IPR00...",10090
2,392,3170,AA3R_MOUSE,Q61618; Q9R202;,MEADNTTETDWLNITYITMEAAIGLCAVVGNMLVIWVVKLNPTLRT...,GO:0004930;GO:0004872;GO:0003674;GO:0004871;GO...,"['IPR000466', 'IPR001634', 'IPR000276', 'IPR01...",10090
3,462,3586,AARE_ARATH,Q84LM4; O23313;,MDSSGTDSAKELHVGLDPTTEEEYATQSKLLQEFINIPSIDKAWIF...,GO:0008233;GO:0017171;GO:0016787;GO:0008236;GO...,"['IPR011042', 'IPR029058', 'IPR011659', 'IPR00...",3702
4,716,4346,ABHD2_HUMAN,P08910; Q53G48; Q53GU0; Q5FVD9; Q8TC79;,MNAMLETPELPAVFDGVKLAAVAAVLYVIVRCLNLKSPTAPPDLYF...,GO:0005488;GO:0016787;GO:0016298;GO:0052689;GO...,"['IPR029058', 'IPR000073', 'IPR000952', 'IPR01...",9606
...,...,...,...,...,...,...,...,...
674,55712,551992,ZN683_HUMAN,Q8IZ20; Q5T141; Q5T146; Q5T147; Q5T149; Q8NEN4;,MKEESAAQLGCCHRPMALGGTGGSLSPSLDFQLFRGDQVFSACRPL...,GO:0044212;GO:0001191;GO:0005515;GO:0005488;GO...,"['IPR007087', 'IPR015880', 'IPR013087']",9606
675,55713,551993,ZN683_MOUSE,I7HJS4;,MKALDGLRESLYPSLDFQLYQDDQVCSADASQPLADSVGAHDLAWS...,GO:0044877;GO:0005488;GO:0003674;GO:0003682,"['IPR007087', 'IPR015880', 'IPR013087']",10090
676,55718,552022,ZN704_MOUSE,Q9ERQ3; Q7TQL8; Q8BJW9;,MQARRLAKRPSLGSRRGGAAPAPAPEAAALGLPPPGPSPAAAPGSW...,GO:0035326;GO:0003690;GO:0044212;GO:0003676;GO...,['IPR007087'],10090
677,55732,552113,ZN809_MOUSE,G3X9G7; Q4KL58; Q8BIJ2;,MGLVSFEDVAVDFTLEEWQDLDAAQRTLYRDVMLETYSSLVFLDPC...,GO:0004527;GO:0016779;GO:0016772;GO:0016740;GO...,"['IPR001909', 'IPR007087', 'IPR015880', 'IPR01...",10090


In [25]:
testing_data = get_training_data(test_df,gap=30)
print(len(testing_data))

0it [00:00, ?it/s]

9913


In [26]:
print('Preparing from scratch...')
testing_data_ngrams = []

for i in tqdm.tqdm(range(len(testing_data))):
    testing_data_ngrams.append([testing_data[i][0],get_ngrams(testing_data[i][1],n=4),testing_data[i][2]])
    
print(len(testing_data_ngrams))

Preparing from scratch...


  0%|          | 0/9913 [00:00<?, ?it/s]

9913


In [27]:
print('Preparing from scratch...')
testing_data_skip_grams = []

for i in tqdm.tqdm(range(len(testing_data))):
    testing_data_skip_grams.append([testing_data[i][0],get_skip_grams(testing_data[i][1],n=4),testing_data[i][2]])
    
print(len(testing_data_skip_grams))

Preparing from scratch...


  0%|          | 0/9913 [00:00<?, ?it/s]

9913


In [28]:
#Considering ngrams and skip grams
X_test_ng = [' '.join(sample[1]) for sample in testing_data_ngrams]
X_test_sg = [' '.join(sample[1]) for sample in testing_data_skip_grams]
y_test = [sample[2] for sample in testing_data]

del testing_data_ngrams
del testing_data_skip_grams

In [29]:
print(len(X_test_ng),len(X_test_sg))

9913 9913


In [31]:
from utils import FUNC_DICT, Ontology, NAMESPACES
NUM_CLASSES = 652

train_df = pd.read_pickle('data_golabeler/train_data.pkl')
test_df = pd.read_pickle('data_golabeler/test_data.pkl')
annotations = train_df['annotations'].values
annotations = list(map(lambda x: set(x), annotations))
test_annotations = test_df['annotations'].values
test_annotations = list(map(lambda x: set(x), test_annotations))

go_rels = Ontology('data_golabeler/go.obo', with_rels=True)
go_rels.calculate_ic(annotations + test_annotations)

In [32]:
def evaluate_annotations(real_annots, pred_annots):
    total = 0
    p = 0.0
    r = 0.0
    p_total= 0
    fps = []
    fns = []
    ru = 0.0
    mi = 0.0
    for i in range(len(real_annots)):
        if len(real_annots[i]) == 0:
            continue
        tp = set(real_annots[i]).intersection(set(pred_annots[i]))
        fp = set(pred_annots[i]) - tp
        fn = set(real_annots[i]) - tp
        for go_id in fp:
            mi += go_rels.get_ic(go_id)
        for go_id in fn:
            ru += go_rels.get_ic(go_id)
        fps.append(fp)
        fns.append(fn)
        tpn = len(tp)
        fpn = len(fp)
        fnn = len(fn)
        total += 1
        recall = tpn / (1.0 * (tpn + fnn))
        r += recall
        if len(pred_annots[i]) > 0:
            p_total += 1
            precision = tpn / (1.0 * (tpn + fpn))
            p += precision
    ru /= total
    mi /= total
    r /= total
    if p_total > 0:
        p /= p_total
    f = 0.0
    if p + r > 0:
        f = 2 * p * r / (p + r)
    s = math.sqrt(ru * ru + mi * mi)
    return f, p, r, fps, fns, s, ru, mi

In [33]:
def compute_metrics(predictions):
    final_predictions = []
    actual_y_test = []

    current_entry = ''
    counter = 0
    total_counts = 0
    start_index = 0

    if len(predictions) == len(testing_data):
        temp = np.zeros(NUM_CLASSES)
        for i in range(len(predictions)):
            if current_entry != testing_data[start_index+i][0]:
                #compute prev
                if i!=0:
                    temp /= counter
                    final_predictions.append(temp)

                #reset
                total_counts += counter
                counter = 1
                temp = np.zeros(NUM_CLASSES)

                #init new
                current_entry = testing_data[start_index+i][0]
                temp += np.array(predictions[i])
                actual_y_test.append(testing_data[start_index+i][2])
            else:
                temp += np.array(predictions[i])
                counter += 1

        total_counts += counter
        temp /= counter
        final_predictions.append(temp)

    else:
        print('Lengths of predictions dont match with test data')
    
    final_predictions = np.array(final_predictions, dtype=float)
    actual_y_test = np.array(actual_y_test, dtype=float)
    
    #Computing maximal F1-score
    fmax = 0.0
    tmax = 0.0
    precisions = []
    recalls = []
    smin = 1000000.0
    rus = []
    mis = []
    
    for t in tqdm.tqdm(range(0, 101)):
        
        threshold = t / 100.0
        pred_annots = []
        real_annots = []
    
        for i in range(len(final_predictions)):
            new_preds = []
            new_ys = []
            for j in range(NUM_CLASSES):
                if final_predictions[i][j]>=threshold:
                    new_preds.append(go_terms_bp[j]) #GO_TERMS_BP
                if actual_y_test[i][j]==1:
                    new_ys.append(go_terms_bp[j])
            pred_annots.append(new_preds)
            real_annots.append(new_ys)
    
        fscore, prec, rec, fps, fns, s, ru, mi = evaluate_annotations(real_annots, pred_annots)
        avg_fp = sum(map(lambda x: len(x), fps)) / len(fps)
        avg_ic = sum(map(lambda x: sum(map(lambda go_id: go_rels.get_ic(go_id), x)), fps)) / len(fps)
        print(f'{avg_fp} {avg_ic}')
        precisions.append(prec)
        recalls.append(rec)
        print(f'Fscore: {fscore}, Precision: {prec}, Recall: {rec} S: {s}, RU: {ru}, MI: {mi} threshold: {threshold}')
        if fmax < fscore:
            fmax = fscore
            tmax = threshold
        if smin > s:
            smin = s
    print('\nFinal Results (Maximal F1-score):')
    print(f'Fmax: {fmax:0.3f}, Smin: {smin:0.3f}, threshold: {tmax}')
    precisions = np.array(precisions)
    recalls = np.array(recalls)
    sorted_index = np.argsort(recalls)
    recalls = recalls[sorted_index]
    precisions = precisions[sorted_index]
    aupr = np.trapz(precisions, recalls)
    print(f'AUPR: {aupr:0.3f}')
        
    
#     rec = recall(actual_y_test,final_predictions)
#     prec = precision(actual_y_test,final_predictions)
#     f1 = f1_score(actual_y_test,final_predictions)
    
#     return rec,prec,f1

In [34]:

recs = []
precs = []
f1s = []

for i in range(1):
    
#     print('Splitting into train-test...')
#     X_train_ng, y_train, X_test_ng, y_test, start_index, prev_index1 = train_test_split(X_ng,y,i,prev_index,Kfolds)
#     X_train_sg, _, X_test_sg, _, _, _= train_test_split(X_sg,y,i,prev_index,Kfolds)
    
#     prev_index = prev_index1
    
    print('Tokenizing...')
    X_train_ng, X_test_ng, vocab_size_ng, tokenizer1 = tokenization(X_train_ng, X_test_ng, MAX_LEN_NG)  
    X_train_sg, X_test_sg, vocab_size_sg, tokenizer2 = tokenization(X_train_sg, X_test_sg, MAX_LEN_SG)
    
   
    print('Shuffling...')   
    shuffled = [[X_train_ng[i],X_train_sg[i],y_train[i]] for i in range(len(X_train_ng))]
    np.random.shuffle(shuffled)

    X_train_ng = [shuffled[i][0] for i in range(len(shuffled))]
    X_train_sg = [shuffled[i][1] for i in range(len(shuffled))]
    y_train = [shuffled[i][2] for i in range(len(shuffled))]
    X_train_ng = np.array(X_train_ng)
    X_train_sg = np.array(X_train_sg)
    y_train = np.array(y_train)
    
    
    model = get_model_ng_sg(vocab_size_ng, vocab_size_sg)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])

    print('Training...')
    history = model.fit([X_train_ng,X_train_sg], y_train, batch_size=32, epochs=10,validation_split=0.2)
    
    print('Evaluating model...')
    predictions = model.predict([X_test_ng,X_test_sg])
    
    print('Computing Metrics...\n')
    compute_metrics(predictions)
    
#     recs.append(rec.numpy())
#     precs.append(prec.numpy())
#     f1s.append(f1.numpy())

# print('Recall:',sum(recs)/len(recs))
# print('Precision:',sum(precs)/len(precs))
# print('F1-Score:',sum(f1s)/len(f1s))

Tokenizing...
Shuffling...


2023-02-01 19:05:32.659168: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0


Training...


2023-02-01 19:05:36.180172: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2399198304 exceeds 10% of free system memory.
2023-02-01 19:05:37.917801: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2399198304 exceeds 10% of free system memory.


Epoch 1/10


...

14375/14375 [==============================] - 578s 39ms/step - loss: 0.2135 - recall: 0.4766 - precision: 0.3897 - f1_score: 0.4267 - val_loss: 0.1866 - val_recall: 0.5235 - val_precision: 0.4164 - val_f1_score: 0.4626
Epoch 2/10
14375/14375 [==============================] - 563s 39ms/step - loss: 0.1623 - recall: 0.5784 - precision: 0.4271 - f1_score: 0.4900 - val_loss: 0.1543 - val_recall: 0.6035 - val_precision: 0.4349 - val_f1_score: 0.5046
Epoch 3/10
14375/14375 [==============================] - 572s 40ms/step - loss: 0.1303 - recall: 0.6548 - precision: 0.4794 - f1_score: 0.5524 - val_loss: 0.1344 - val_recall: 0.6448 - val_precision: 0.4994 - val_f1_score: 0.5618
Epoch 4/10
14375/14375 [==============================] - 575s 40ms/step - loss: 0.1064 - recall: 0.7221 - precision: 0.5349 - f1_score: 0.6135 - val_loss: 0.1197 - val_recall: 0.7004 - val_precision: 0.5279 - val_f1_score: 0.6010
Epoch 5/10
14375/14375 [==============================] - 575s 40ms/step - loss: 0.0887

  0%|          | 0/101 [00:00<?, ?it/s]

642.5905743740796 1234.5234915187455
Fscore: 0.028452650540974435, Precision: 0.014431634395583539, Recall: 1.0 S: 1234.5234915186497, RU: 0.0, MI: 1234.5234915186497 threshold: 0.0
110.61855670103093 225.58672493776353
Fscore: 0.13713553038631346, Precision: 0.07456892382755981, Recall: 0.8520062160001708 S: 225.60000217480157, RU: 2.447552476108176, MI: 225.58672493776572 threshold: 0.01
76.71281296023564 142.5605632065176
Fscore: 0.17830285724055342, Precision: 0.1001461502414523, Recall: 0.8120416935023513 S: 142.59423594455382, RU: 3.0987001874582916, MI: 142.560563206517 threshold: 0.02
60.44182621502209 106.2774415177051
Fscore: 0.2100198084787348, Precision: 0.12110049777414421, Recall: 0.790322100121274 S: 106.33400432789908, RU: 3.4678380667629765, MI: 106.27744151770106 threshold: 0.03
50.34756995581738 85.24928994671515
Fscore: 0.23642561986338265, Precision: 0.13941906792714148, Recall: 0.7771818951396704 S: 85.32871997514599, RU: 3.6808988275139396, MI: 85.24928994671149 

4.394698085419735 5.193631442791221
Fscore: 0.5564413008186877, Precision: 0.565277908050197, Recall: 0.5478767146745591 S: 8.53241857834973, RU: 6.76966463221172, MI: 5.193631442791198 threshold: 0.4
4.2341678939617085 4.959574650128183
Fscore: 0.5586058172276288, Precision: 0.573900148332341, Recall: 0.5441055076967917 S: 8.427043000454603, RU: 6.813051667301293, MI: 4.959574650128171 threshold: 0.41
4.045655375552283 4.7141736355091615
Fscore: 0.5618235486187743, Precision: 0.5846879697081014, Recall: 0.5406800713020948 S: 8.321727151756361, RU: 6.8576752418403055, MI: 4.714173635509149 threshold: 0.42
3.898379970544919 4.522282169834803
Fscore: 0.5630947307848053, Precision: 0.592541186176711, Recall: 0.5364364135396984 S: 8.261255338852237, RU: 6.913559412495772, MI: 4.522282169834796 threshold: 0.43
3.727540500736377 4.288102450995912
Fscore: 0.5656388554818158, Precision: 0.6037925428589651, Recall: 0.5320204509427255 S: 8.17594222315551, RU: 6.96119304474024, MI: 4.288102450995

0.9204712812960235 0.8414609451364234
Fscore: 0.502043295911881, Precision: 0.8569550305935144, Recall: 0.35501305946760064 S: 8.681578399712956, RU: 8.640702922110718, MI: 0.8414609451364236 threshold: 0.8
0.8792341678939617 0.8071415941393397
Fscore: 0.49346794505620534, Precision: 0.8603102482632351, Recall: 0.3459517154673327 S: 8.740983117618367, RU: 8.703637648104355, MI: 0.8071415941393404 threshold: 0.81
0.8247422680412371 0.7514218664539972
Fscore: 0.4880668286548922, Precision: 0.8656515474011502, Recall: 0.3398354898136352 S: 8.803601657352626, RU: 8.771474637708113, MI: 0.7514218664539974 threshold: 0.82
0.7643593519882179 0.7050059157919096
Fscore: 0.48242227255350106, Precision: 0.8700269200008037, Recall: 0.33373873143243094 S: 8.863860659467493, RU: 8.835779108214165, MI: 0.7050059157919099 threshold: 0.83
0.7231222385861561 0.6643055948433997
Fscore: 0.4761763086719848, Precision: 0.8768158492940551, Recall: 0.3268364107253466 S: 8.924412770317703, RU: 8.89965400291884

In [35]:
print(len(predictions))

9913


In [36]:
model.save('mf_model.h5')

In [37]:
np.save('mf_predictions',predictions)

In [70]:
predictions = np.load('mf_predictions.npy')
print(len(predictions))

35365
